In [12]:
import pandas as pd
import json
from io import StringIO
from pathlib import Path

# ---------- 通用：把 _value 从十六进制还原成 Python 对象 ----------
def decode_value(hex_like: str):
    """把看起来像 hex 的字符串解成 JSON（失败返回 None）"""
    try:
        # 仅保留 0-9a-fA-F，去掉换行和控制字符
        cleaned = ''.join(ch for ch in str(hex_like) if ch.isalnum())
        b = bytes.fromhex(cleaned)
        s = b.decode('utf-8', errors='ignore')
        return json.loads(s)
    except Exception:
        return None

# ---------- 1) 读取与解析 TomTom ----------
tomtom_raw = pd.read_parquet(
    'data/20250820163000_stream.tomtom.analyze-sail.parquet', engine='fastparquet'
)

# 把 _value 解出来（得到包含 time 和 data 的字典）
tomtom_decoded = tomtom_raw['_value'].map(decode_value)

# 只保留解码成功的
tomtom_decoded = [d for d in tomtom_decoded if isinstance(d, dict)]

# 每条记录的 d['data'] 是一段 CSV 文本；逐条读成 DataFrame 并加上 time 列
frames = []
for d in tomtom_decoded:
    time_str = d.get('time')  # 例如 '2025-08-20T08:37:13.722331+02:00'
    csv_blob = d.get('data', '')
    if not csv_blob:
        continue
    df_piece = pd.read_csv(StringIO(csv_blob))  # 有表头：id,traffic_level
    df_piece['time'] = pd.to_datetime(time_str)  # 加时间戳
    frames.append(df_piece)

tomtom = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame(columns=['id','traffic_level','time'])

# 排序 & 简单预览
tomtom = tomtom.sort_values('time').reset_index(drop=True)
print('TomTom tidy shape:', tomtom.shape)
print(tomtom.head(10))

# 可选：看看时间范围、id 数量
if not tomtom.empty:
    print('time range:', tomtom['time'].min(), '->', tomtom['time'].max())
    print('unique links:', tomtom['id'].nunique())

# ---------- 2) 读取与解析 Vessel ----------
import re, json, binascii

def decode_value_fuzzy(val):
    """
    尝试把 Kafka/FME 风格的 _value 解成 dict：
    1) 优先按十六进制解析成 bytes
    2) 从 bytes 里截取第一段 {...} 的 JSON 子串
    3) 解析 JSON，失败则返回 None
    """
    if val is None:
        return None
    s = str(val).strip()

    # 先尽量取出十六进制（允许换行/空格）
    hex_candidate = re.sub(r'\s+', '', s)
    is_hex = bool(re.fullmatch(r'[0-9a-fA-F]+', hex_candidate)) and (len(hex_candidate) % 2 == 0)

    b = None
    if is_hex:
        try:
            b = bytes.fromhex(hex_candidate)
        except binascii.Error:
            b = None

    # 如果不是纯 hex，就当作原始 utf-8 文本
    if b is None:
        b = s.encode('utf-8', errors='ignore')

    # 在字节流里找 JSON 子串
    lb = b.find(b'{')
    rb = b.rfind(b'}')
    if lb != -1 and rb != -1 and rb > lb:
        js = b[lb:rb+1].decode('utf-8', errors='ignore')
        try:
            return json.loads(js)
        except Exception:
            return None
    return None

# ---------- 解析 Vessel ----------
vessel_raw = pd.read_parquet(
    'data/20250820163000_stream.vessel-positions-anonymized-processed.analyze-sail.parquet',
    engine='fastparquet'
)

vessel_decoded = [decode_value_fuzzy(x) for x in vessel_raw['_value']]
vessel_decoded = [d for d in vessel_decoded if isinstance(d, dict)]

# 展平（用 sep='.' 让嵌套键变成 position.lat 这种平面列）
vessel = pd.json_normalize(vessel_decoded, sep='.')

# 统一时间列（按你实际的键名，自行增加）
for col in ['time', 'timestamp', 'ts', 'observed_at', 'event_time']:
    if col in vessel.columns:
        vessel[col] = pd.to_datetime(vessel[col], errors='coerce')

print('Vessel tidy shape:', vessel.shape)
print(vessel.head(10))

# 快速猜测经纬度列名，方便你确认
for lat_key in ['lat','latitude','position.lat','pos.lat','geom.lat','location.lat']:
    for lon_key in ['lon','longitude','position.lon','pos.lon','geom.lon','location.lon']:
        if lat_key in vessel.columns and lon_key in vessel.columns:
            print(f'Found lat/lon columns: {lat_key}, {lon_key}')
            print(vessel[[lat_key, lon_key]].head())
            break



# 可选：挑一组可能的经纬度列名，便于你快速确认
for lat_key in ['lat','latitude','position.lat','pos.lat','geom.lat']:
    for lon_key in ['lon','longitude','position.lon','pos.lon','geom.lon']:
        if lat_key in vessel.columns and lon_key in vessel.columns:
            print(f'Found lat/lon columns: {lat_key}, {lon_key}')
            print(vessel[[lat_key, lon_key]].head())
            break


TomTom tidy shape: (5177644, 3)
          id  traffic_level                             time
0  213399051           0.49 2025-08-20 08:37:13.722331+02:00
1  247378050           1.00 2025-08-20 08:37:13.722331+02:00
2  247379106           1.00 2025-08-20 08:37:13.722331+02:00
3  247380059           1.00 2025-08-20 08:37:13.722331+02:00
4  247380066           1.00 2025-08-20 08:37:13.722331+02:00
5  247381056           1.00 2025-08-20 08:37:13.722331+02:00
6  248367009           1.00 2025-08-20 08:37:13.722331+02:00
7  248370083           1.00 2025-08-20 08:37:13.722331+02:00
8  247378009           1.00 2025-08-20 08:37:13.722331+02:00
9  248372134           1.00 2025-08-20 08:37:13.722331+02:00
time range: 2025-08-20 08:37:13.722331+02:00 -> 2025-08-20 16:29:56.126985+02:00
unique links: 10742
Vessel tidy shape: (1059291, 33)
   identifier-sensor  radar-length-cm  radar-beam-cm  position-x  position-y  \
0                251             2500           1200    100561.0    497650.0   
1  

In [13]:
import pandas as pd
import numpy as np

# ---------- 1) 基本类型清洗 ----------
# TomTom
tomtom['id'] = tomtom['id'].astype('int64')
tomtom['traffic_level'] = tomtom['traffic_level'].astype('float32')
# 统一时区到 UTC（TomTom 原本是 +02:00）
tomtom['time'] = pd.to_datetime(tomtom['time'], utc=True)

# Vessel
for col in ['upload-timestamp', 'time', 'timestamp', 'ts', 'observed_at', 'event_time']:
    if col in vessel.columns:
        vessel[col] = pd.to_datetime(vessel[col], utc=True, errors='coerce')
# 选一个主时间列（这批数据里有 'upload-timestamp'）
vessel['time'] = vessel['upload-timestamp']

# 经纬度与数值列
for c in ['lat','lon','speed-in-centimeters-per-second','x-speed','y-speed','orientation']:
    if c in vessel.columns:
        vessel[c] = pd.to_numeric(vessel[c], errors='coerce')

# ---------- 2) 统一时间粒度（示例：1 分钟；如需 3 分钟改成 '3min'） ----------
BIN = '1min'   # 或者 '3min'
tomtom['time_bin'] = tomtom['time'].dt.floor(BIN)
vessel['time_bin'] = vessel['time'].dt.floor(BIN)

# ---------- 3) 快速聚合 ----------
# TomTom：每分钟每 link 的平均 traffic_level（也可改成中位数/最后一个值）
tt_min = (tomtom
          .groupby(['time_bin','id'], as_index=False)['traffic_level']
          .mean())

# Vessel：每分钟的船舶数量（全域/也可按网格、缓冲区等聚合）
ves_min = (vessel
           .groupby('time_bin', as_index=False)
           .size()
           .rename(columns={'size':'vessel_count'}))

print('tt_min shape:', tt_min.shape)
print('ves_min shape:', ves_min.shape)
print(tt_min.head(), '\n', ves_min.head())

# ---------- 4) 关联（按 time_bin 左连接，得到 traffic 和船数同窗对齐） ----------
tt_ves = tt_min.merge(ves_min, on='time_bin', how='left')
print('tt_ves shape:', tt_ves.shape)
print(tt_ves.head())

# ---------- 5) 可选：保存中间结果（后续建模更快） ----------
tt_min.to_parquet('data/tt_minute.parquet', index=False)
ves_min.to_parquet('data/vessel_minute.parquet', index=False)
tt_ves.to_parquet('data/tt_vessel_minute.parquet', index=False)


tt_min shape: (4210864, 3)
ves_min shape: (481, 2)
                   time_bin         id  traffic_level
0 2025-08-20 06:37:00+00:00  202394085          0.665
1 2025-08-20 06:37:00+00:00  202394096          0.435
2 2025-08-20 06:37:00+00:00  202394097          0.000
3 2025-08-20 06:37:00+00:00  203394034          0.760
4 2025-08-20 06:37:00+00:00  203394046          0.760 
                    time_bin  vessel_count
0 2025-08-20 06:28:00+00:00             3
1 2025-08-20 06:29:00+00:00             9
2 2025-08-20 06:30:00+00:00             7
3 2025-08-20 06:31:00+00:00            15
4 2025-08-20 06:32:00+00:00             8
tt_ves shape: (4210864, 4)
                   time_bin         id  traffic_level  vessel_count
0 2025-08-20 06:37:00+00:00  202394085          0.665        2309.0
1 2025-08-20 06:37:00+00:00  202394096          0.435        2309.0
2 2025-08-20 06:37:00+00:00  202394097          0.000        2309.0
3 2025-08-20 06:37:00+00:00  203394034          0.760        2309.0
4 20

In [14]:
# --- 1) TomTom 全体平均 & Top-N 方差最大的路段 ---
# 全体平均
tt_all = (tt_min.groupby('time_bin', as_index=False)['traffic_level']
          .mean().rename(columns={'traffic_level':'tt_mean'}))

# Top-N 波动最大的 link
N = 200
var_by_id = (tt_min.groupby('id')['traffic_level']
             .var().sort_values(ascending=False).head(N).index)
tt_top = (tt_min[tt_min['id'].isin(var_by_id)]
          .groupby('time_bin', as_index=False)['traffic_level']
          .mean().rename(columns={'traffic_level':'tt_topN_mean'}))

# --- 2) 与船舶每分钟强度对齐 ---
x = ves_min.merge(tt_all, on='time_bin', how='outer').merge(tt_top, on='time_bin', how='outer')
x = x.sort_values('time_bin')
cols = ['vessel_count','tt_mean','tt_topN_mean']
x[cols] = x[cols].ffill()

# --- 3) 计算同时 & 滞后相关（例如滞后 0~30 分钟） ---
def lag_corr(s1, s2, max_lag=30):
    out = []
    for k in range(0, max_lag+1):
        c = s1.corr(s2.shift(k), method='spearman')  # 对秩更稳健
        out.append((k, c))
    return pd.DataFrame(out, columns=['lag_min','corr'])

lag_all = lag_corr(x['vessel_count'], x['tt_mean'], 30)
lag_top = lag_corr(x['vessel_count'], x['tt_topN_mean'], 30)

print(lag_all.head(), '\n', lag_top.head())
# 你可以看最大相关对应的滞后分钟数，判断因果方向的线索


   lag_min      corr
0        0 -0.435692
1        1 -0.444113
2        2 -0.444638
3        3 -0.451511
4        4 -0.454839 
    lag_min      corr
0        0 -0.452978
1        1 -0.453215
2        2 -0.452641
3        3 -0.448329
4        4 -0.452173


In [15]:
# 设定研究范围（用数据的 min/max 也行）
lat_min, lat_max = vessel['lat'].quantile([0.01, 0.99])
lon_min, lon_max = vessel['lon'].quantile([0.01, 0.99])

# 定义网格大小（约 250 m，可按需要调）：经纬度粗换算 ~1度≈111km
cell_deg = 0.002  # ~ 200m–250m 在阿姆斯特丹
vessel['lat_bin'] = ((vessel['lat'] - lat_min) // cell_deg).astype('Int64')
vessel['lon_bin'] = ((vessel['lon'] - lon_min) // cell_deg).astype('Int64')

grid = (vessel
        .dropna(subset=['lat_bin','lon_bin','time_bin'])
        .groupby(['time_bin','lat_bin','lon_bin'], as_index=False)
        .size().rename(columns={'size':'vessel_cnt'}))

print('grid shape:', grid.shape)
print(grid.head())
# 这张表就是“每分钟 × 网格”的船舶密度；之后有了道路几何，只需要把每条路段的缓冲区落到若干网格汇总即可。


grid shape: (270831, 4)
                   time_bin  lat_bin  lon_bin  vessel_cnt
0 2025-08-20 06:28:00+00:00      203      772           1
1 2025-08-20 06:28:00+00:00      226      679           1
2 2025-08-20 06:28:00+00:00      668      826           1
3 2025-08-20 06:29:00+00:00       -5      307           1
4 2025-08-20 06:29:00+00:00       50       89           1


In [17]:
def lag_corr_series(s1, s2, max_lag=60, method='spearman'):
    out = []
    for k in range(0, max_lag+1):
        out.append((k, s1.corr(s2.shift(k), method=method)))
    return pd.DataFrame(out, columns=['lag_min','corr'])

lag_all = lag_all.rename(columns={'corr': 'rho'})
lag_top = lag_top.rename(columns={'corr': 'rho'})

best_all = lag_all.loc[lag_all['rho'].abs().idxmax()]
best_top = lag_top.loc[lag_top['rho'].abs().idxmax()]

print('ALL  best lag:', int(best_all['lag_min']), 'rho=', round(float(best_all['rho']), 3))
print('TopN best lag:', int(best_top['lag_min']), 'rho=', round(float(best_top['rho']), 3))


ALL  best lag: 58 rho= -0.533
TopN best lag: 58 rho= -0.477


In [18]:
def lag_corr_bidir(s1, s2, max_lag=90, method='spearman'):
    # dirA: vessel leads -> corr( vessel(t), tt(t+k) )
    A = [(k, s1.corr(s2.shift(k), method=method)) for k in range(max_lag+1)]
    A = pd.DataFrame(A, columns=['lag_min','rho_A'])  # vessel -> tt

    # dirB: tt leads -> corr( tt(t), vessel(t+k) )
    B = [(k, s2.corr(s1.shift(k), method=method)) for k in range(max_lag+1)]
    B = pd.DataFrame(B, columns=['lag_min','rho_B'])  # tt -> vessel

    return A.merge(B, on='lag_min')

lag_bi = lag_corr_bidir(x['vessel_count'], x['tt_mean'], max_lag=90)
print(lag_bi.loc[lag_bi['rho_A'].abs().idxmax()])
print(lag_bi.loc[lag_bi['rho_B'].abs().idxmax()])


lag_min    79.000000
rho_A      -0.573713
rho_B      -0.461175
Name: 79, dtype: float64
lag_min    31.000000
rho_A      -0.505043
rho_B      -0.489442
Name: 31, dtype: float64


In [19]:
# 15分钟滚动去均值
x_dt = x[['time_bin','vessel_count','tt_mean']].copy()
x_dt['vessel_detr'] = x_dt['vessel_count'] - x_dt.set_index('time_bin')['vessel_count'].rolling('15min').mean().values
x_dt['tt_detr']     = x_dt['tt_mean']     - x_dt.set_index('time_bin')['tt_mean'].rolling('15min').mean().values

lag_dt = [(k, x_dt['vessel_detr'].corr(x_dt['tt_detr'].shift(k), method='spearman')) for k in range(91)]
lag_dt = pd.DataFrame(lag_dt, columns=['lag_min','rho_detr'])
print(lag_dt.loc[lag_dt['rho_detr'].abs().idxmax()])


lag_min     79.000000
rho_detr    -0.104362
Name: 79, dtype: float64


In [24]:
import pandas as pd
import numpy as np

# --- 1) 统一到 3 分钟粒度（更贴近 TomTom 原节拍），并用 inner join 避免 ffill 引入趋势 ---
BIN = '3min'
tomtom['time_bin'] = tomtom['time'].dt.floor(BIN)
vessel['time_bin'] = vessel['time'].dt.floor(BIN)

tt_min = tomtom.groupby(['time_bin','id'], as_index=False)['traffic_level'].mean()
ves_min = vessel.groupby('time_bin', as_index=False).size().rename(columns={'size':'vessel_count'})

# 全体平均 traffic
tt_all = tt_min.groupby('time_bin', as_index=False)['traffic_level'].mean() \
               .rename(columns={'traffic_level':'tt_mean'})

# 保持“公共时间交集”，避免 ffill
x = pd.merge(tt_all, ves_min, on='time_bin', how='inner').sort_values('time_bin').reset_index(drop=True)

# 若存在缺口，用正规重采样+插值/短窗口均值（可选）
# x = x.set_index('time_bin').resample(BIN).mean().interpolate(limit=2).dropna().reset_index()

# --- 2) 滞后相关：正确标注分钟单位 ---
FREQ_MIN = pd.to_timedelta(BIN).seconds // 60  # 3
def lag_corr_series(s1, s2, max_lag=90, method='spearman'):
    rows = []
    for k in range(max_lag+1):
        rows.append((k, k*FREQ_MIN, s1.corr(s2.shift(k), method=method)))
    return pd.DataFrame(rows, columns=['lag_k','lag_min','rho'])

lag3 = lag_corr_series(x['vessel_count'], x['tt_mean'], 90)
best3 = lag3.loc[lag3['rho'].abs().idxmax()]
print('3-min BIN best lag_k:', int(best3['lag_k']),
      '≈', int(best3['lag_min']), 'min, rho=', round(float(best3['rho']),3))

# --- 3) 去趋势 + 小时固定效应 检查 ---
# 去趋势：滚动 15 分钟窗口（=5个3min样本）
roll = f'{5*FREQ_MIN}min'  # '15min'
xx = x.copy()
xx['vessel_detr'] = xx['vessel_count'] - xx.set_index('time_bin')['vessel_count'].rolling(roll).mean().values
xx['tt_detr']     = xx['tt_mean']      - xx.set_index('time_bin')['tt_mean'].rolling(roll).mean().values

lag_dt = lag_corr_series(xx['vessel_detr'], xx['tt_detr'], 90)
best_dt = lag_dt.loc[lag_dt['rho'].abs().idxmax()]
print('Detrended best lag ≈', int(best_dt['lag_min']), 'min, rho=', round(float(best_dt['rho']),3))

# 小时固定效应 + 滞后（sklearn 近似版）
from sklearn.linear_model import LinearRegression
df = x[['time_bin','tt_mean','vessel_count']].dropna().copy()
df['hour'] = df['time_bin'].dt.tz_convert('Europe/Amsterdam').dt.hour

LAG_K = int(best3['lag_k'])  # 用上面找到的滞后（k 步），你也可以手动填 58/79 做对照
df[f'vessel_lag_k'] = df['vessel_count'].shift(LAG_K)
df = df.dropna()

H = pd.get_dummies(df['hour'].astype('category'), drop_first=True)

X0 = H.values
y  = df['tt_mean'].values
m0 = LinearRegression().fit(X0, y)
r2_0 = m0.score(X0, y)

X1 = np.column_stack([H.values, df['vessel_lag_k'].values])
m1 = LinearRegression().fit(X1, y)
r2_1 = m1.score(X1, y)

print(f'ΔR² = {r2_1 - r2_0:.4f}, coef(lag_k={LAG_K}, ~{LAG_K*FREQ_MIN}min) = {m1.coef_[-1]:.6f}')


3-min BIN best lag_k: 89 ≈ 267 min, rho= 0.806
Detrended best lag ≈ 120 min, rho= -0.215
ΔR² = 0.0051, coef(lag_k=89, ~267min) = -0.000001


ΔR² = 0.0061, coef(lag79) = -0.0000


asof lag0 spearman = -0.4812036724525778
